In [73]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [74]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Mock_ML_Open.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain/Loss_Open
0,1981-02-11,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,-0.003348,Loss
1,1981-02-12,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,Loss
2,1981-02-13,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,Loss
3,1981-02-17,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,Gain
4,1981-02-18,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,Gain


In [75]:
# Convert date into an integer since the system was not taking it as a float so that the ML model can read the date - for now.
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
apple_df['Date'] = apple_df['Date'].astype(int)

In [76]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain/Loss_Open,Season_Spring,Season_Winter,Quarter_Q1,Quarter_Q2
0,350697600000000000,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,-0.003348,Loss,0,1,1,0
1,350784000000000000,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,Loss,0,1,1,0
2,350870400000000000,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,Loss,0,1,1,0
3,351216000000000000,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,Gain,0,1,1,0
4,351302400000000000,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,Gain,0,1,1,0


In [77]:
# Define features set
X = apple_binary_encoded.copy()
X = X.drop("Gain/Loss_Open", axis=1)
X.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Season_Spring,Season_Winter,Quarter_Q1,Quarter_Q2
0,350697600000000000,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,-0.003348,0,1,1,0
1,350784000000000000,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,0,1,1,0
2,350870400000000000,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,0,1,1,0
3,351216000000000000,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,0,1,1,0
4,351302400000000000,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,0,1,1,0


In [78]:
# Define the target set.
y = apple_df["Gain/Loss_Open"].ravel()
y[:5]

array(['Loss', 'Loss', 'Loss', 'Gain', 'Gain'], dtype=object)

In [79]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [80]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [81]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [82]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [83]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [84]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,5,0
Actual 1,1,5


In [85]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [86]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,5,0
Actual 1,1,5


Accuracy Score : 0.9090909090909091
Classification Report
              precision    recall  f1-score   support

        Gain       0.83      1.00      0.91         5
        Loss       1.00      0.83      0.91         6

    accuracy                           0.91        11
   macro avg       0.92      0.92      0.91        11
weighted avg       0.92      0.91      0.91        11



In [87]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.04173535, 0.073711  , 0.0816376 , 0.08225564, 0.09225861,
       0.07235839, 0.06529606, 0.47217411, 0.0072676 , 0.00493494,
       0.00411349, 0.00225722])

In [88]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.47217410929605497, 'Open_24hr_Change'),
 (0.09225860783943361, 'Close'),
 (0.08225564146325351, 'Low'),
 (0.08163759643967948, 'High'),
 (0.07371099701388581, 'Open'),
 (0.07235838895862103, 'Adj Close'),
 (0.06529606404074022, 'Volume'),
 (0.041735352847801964, 'Date'),
 (0.007267596886133957, 'Season_Spring'),
 (0.00493494137390367, 'Season_Winter'),
 (0.004113488384236685, 'Quarter_Q1'),
 (0.0022572154562551018, 'Quarter_Q2')]